In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
import transformers
import torch
import os
import nltk
import pandas as pd
import torch
import numpy as np
from jinja2 import Template
import pickle
from collections import defaultdict

import sys
sys.path.append('/scratch/users/bozyurt20/hpc_run/utilities')
sys.path.append("/scratch/users/bozyurt20/hpc_run/blobs/")
from util_research import *

max_len = 512
num_layers = 24
d_model = 4096

tokenizer = AutoTokenizer.from_pretrained("bigscience/T0pp", truncation_side="right", add_prefix_space=True)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [2]:
model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/T0pp", output_attentions=True, device_map="balanced")
model.eval()

In [44]:
babi_sets = []

for i in range(1, 21):
    babi_sets.append(load_dataset('babi_qa', type='en-10k', task_no='qa' + str(i)))

Using custom data configuration en-10k-qa1-task_no=qa1,type=en-10k
Found cached dataset babi_qa (/kuacc/users/bozyurt20/.cache/huggingface/datasets/babi_qa/en-10k-qa1-task_no=qa1,type=en-10k/1.2.0/a3c1ff28a1e13b872a34a5aae4a0d6715f4ff95cbc823c7d2c04f278e912dfab)


  0%|          | 0/2 [00:00<?, ?it/s]

Using custom data configuration en-10k-qa2-task_no=qa2,type=en-10k


Generating train split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/200 [00:00<?, ? examples/s]

...

Using custom data configuration en-10k-qa20-task_no=qa20,type=en-10k


Generating train split:   0%|          | 0/933 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/93 [00:00<?, ? examples/s]

Dataset babi_qa downloaded and prepared to /kuacc/users/bozyurt20/.cache/huggingface/datasets/babi_qa/en-10k-qa20-task_no=qa20,type=en-10k/1.2.0/a3c1ff28a1e13b872a34a5aae4a0d6715f4ff95cbc823c7d2c04f278e912dfab. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [52]:
for i, babi in enumerate(babi_sets):
    print(i+1)
    print(babi["train"][0]["story"]["text"])
    print()
    print()

1
['Mary moved to the bathroom.', 'John went to the hallway.', 'Where is Mary?', 'Daniel went back to the hallway.', 'Sandra moved to the garden.', 'Where is Daniel?', 'John moved to the office.', 'Sandra journeyed to the bathroom.', 'Where is Daniel?', 'Mary moved to the hallway.', 'Daniel travelled to the office.', 'Where is Daniel?', 'John went back to the garden.', 'John moved to the bedroom.', 'Where is Sandra?']


2
['Mary moved to the bathroom.', 'Sandra journeyed to the bedroom.', 'Mary got the football there.', 'John went to the kitchen.', 'Mary went back to the kitchen.', 'Mary went back to the garden.', 'Where is the football?', 'Sandra went back to the office.', 'John moved to the office.', 'Sandra journeyed to the hallway.', 'Daniel went back to the kitchen.', 'Mary dropped the football.', 'John got the milk there.', 'Where is the football?', 'Mary took the football there.', 'Sandra picked up the apple there.', 'Mary travelled to the hallway.', 'John journeyed to the kitch

In [57]:
num_ex = len(babi_sets[0]["train"])

In [64]:
all_stories = []
for i in range(num_ex):
    story = babi_sets[0]["train"][i]["story"]["text"]
    story = " ".join(story)
    all_stories.append(story)


In [65]:
one_big_story = " ".join(all_stories)
one_big_story

'Mary moved to the bathroom. John went to the hallway. Where is Mary? Daniel went back to the hallway. Sandra moved to the garden. Where is Daniel? John moved to the office. Sandra journeyed to the bathroom. Where is Daniel? Mary moved to the hallway. Daniel travelled to the office. Where is Daniel? John went back to the garden. John moved to the bedroom. Where is Sandra? Sandra travelled to the office. Sandra went to the bathroom. Where is Sandra? Mary went to the bedroom. Daniel moved to the hallway. Where is Sandra? John went to the garden. John travelled to the office. Where is Sandra? Daniel journeyed to the bedroom. Daniel travelled to the hallway. Where is John? John went to the bedroom. John travelled to the office. Where is Daniel? Mary went to the bedroom. John journeyed to the bathroom. Where is John? Sandra journeyed to the hallway. John journeyed to the garden. Where is Mary? John journeyed to the bathroom. Sandra journeyed to the garden. Where is John? Sandra went back to

In [67]:
unique_words = list(set(one_big_story.split(" ")))
len(unique_words)

23

In [68]:
unique_words

['Sandra?',
 'moved',
 'hallway.',
 'bedroom.',
 'Mary?',
 'John',
 'office.',
 'garden.',
 'is',
 'Daniel?',
 'the',
 'Mary',
 'back',
 'Daniel',
 'bathroom.',
 'journeyed',
 'John?',
 'Sandra',
 'to',
 'travelled',
 'went',
 'kitchen.',
 'Where']

In [40]:
for key, value in babi_17["train"][444]["story"].items():
    
    print(len(value))
    print()

10

10

10

10

10



In [16]:
babi["train"]["story"][0]["text"]

['Mary moved to the bathroom.',
 'John went to the hallway.',
 'Where is Mary?',
 'Daniel went back to the hallway.',
 'Sandra moved to the garden.',
 'Where is Daniel?',
 'John moved to the office.',
 'Sandra journeyed to the bathroom.',
 'Where is Daniel?',
 'Mary moved to the hallway.',
 'Daniel travelled to the office.',
 'Where is Daniel?',
 'John went back to the garden.',
 'John moved to the bedroom.',
 'Where is Sandra?']

# Old Stuff

In [108]:
my_str = "John travelled to London."
input_ids = tokenizer.encode(my_str, return_tensors="pt")
input_ids = input_ids.to(model.encoder.device)
with torch.no_grad():
    encoder_outputs = model.encoder(input_ids, output_hidden_states=True)
hidden_states = encoder_outputs.hidden_states

new code working-modeling_t5


In [14]:
nu = 3 * encoder_outputs.last_hidden_state.std().item()
nu

0.17141779884696007

In [125]:
my_str = "John travelled to London. Where did John travel to?"
input_ids = tokenizer.encode(my_str, return_tensors="pt")
input_ids = input_ids.to(model.encoder.device)
with torch.no_grad():
    encoder_outputs = model.encoder(input_ids, output_hidden_states=True)
hidden_states = encoder_outputs.hidden_states

new code working-modeling_t5


In [110]:
B, T, d = encoder_outputs.last_hidden_state.shape
model_outs = []
decoder_input_ids = tokenizer.encode('<pad>', add_special_tokens=False, return_tensors="pt").cuda()

for i in range(T):
    encoder_out_copy = copy.deepcopy(encoder_outputs)
    encoder_out_copy.last_hidden_state[0, i, :] += 0.1#nu * torch.from_numpy(prng.randn(d)).cuda()
    
    model_out = model(decoder_input_ids=decoder_input_ids, 
                      encoder_outputs=encoder_out_copy)
    
    model_outs.append(model_out)

new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5


In [107]:
import copy

In [133]:
B, T, d = encoder_outputs.last_hidden_state.shape
model_outs2 = []
decoder_input_ids = tokenizer.encode('<pad>', add_special_tokens=False, return_tensors="pt").cuda()

for i in range(T):
    encoder_out_copy = copy.deepcopy(encoder_outputs)
    encoder_out_copy.last_hidden_state[0, i, :] += nu * torch.from_numpy(prng.randn(d)).cuda()
    
    model_out = model.generate(decoder_input_ids=decoder_input_ids, 
                               encoder_outputs=encoder_out_copy,
                               return_dict_in_generate=True,
                               max_new_tokens=1,
                               output_scores=True)
    
    model_outs2.append(model_out)

new code working-utils
llama generation happening.
new code working-modeling_t5
new code working-utils
llama generation happening.
new code working-modeling_t5
new code working-utils
llama generation happening.
new code working-modeling_t5
new code working-utils
llama generation happening.
new code working-modeling_t5
new code working-utils
llama generation happening.
new code working-modeling_t5
new code working-utils
llama generation happening.
new code working-modeling_t5
new code working-utils
llama generation happening.
new code working-modeling_t5
new code working-utils
llama generation happening.
new code working-modeling_t5
new code working-utils
llama generation happening.
new code working-modeling_t5
new code working-utils
llama generation happening.
new code working-modeling_t5
new code working-utils
llama generation happening.
new code working-modeling_t5
new code working-utils
llama generation happening.
new code working-modeling_t5
new code working-utils
llama generation 

In [134]:
tokens = iter(tokenizer.tokenize(my_str) + ["eos"])
for out in model_outs2:
    next_token_scores = torch.nn.functional.softmax(
                        out.scores[0].float(), dim=-1
                    )  # (batch_size * num_beams, vocab_size)
    probability_correct = next_token_scores[0][tokenizer.encode("London")[0]].item()
    token = next(tokens)
    print(token, probability_correct)

▁John 0.7089988589286804
▁ 0.7796667814254761
travelled 0.5765650272369385
▁to 0.6640745401382446
▁London 0.6147871017456055
. 0.7720080018043518
▁Where 0.6802257895469666
▁did 0.7699546217918396
▁John 0.7249479293823242
▁travel 0.719036340713501
▁to 0.6139716506004333
? 0.014809751883149147
eos 0.006303423084318638


In [132]:
tokens = iter(tokenizer.tokenize(my_str) + ["eos"])
for out in model_outs2:
    next_token_scores = torch.nn.functional.softmax(
                        out.scores[0].float(), dim=-1
                    )  # (batch_size * num_beams, vocab_size)
    probability_correct = next_token_scores[0][tokenizer.encode("London")[0]].item()
    token = next(tokens)
    print(token, probability_correct)

▁John 0.7547619342803955
▁ 0.7547619342803955
travelled 0.7547619342803955
▁to 0.7547619342803955
▁London 0.7547619342803955
. 0.7547619342803955
▁Where 0.7547619342803955
▁did 0.7547619342803955
▁John 0.7547619342803955
▁travel 0.7547619342803955
▁to 0.7547619342803955
? 0.7547619342803955
eos 0.7547619342803955


In [23]:
torch.from_numpy(prng.randn(10000000)).std()

tensor(0.9999, dtype=torch.float64)

In [ ]:
encoder_outputs.last_hidden_state[0, i, :] += nu * torch.from_numpy(prng.randn(d))

In [6]:
for layer_hidden_states in hidden_states:
    print(layer_hidden_states.shape)

torch.Size([1, 7, 4096])
torch.Size([1, 7, 4096])
torch.Size([1, 7, 4096])
torch.Size([1, 7, 4096])
torch.Size([1, 7, 4096])
torch.Size([1, 7, 4096])
torch.Size([1, 7, 4096])
torch.Size([1, 7, 4096])
torch.Size([1, 7, 4096])
torch.Size([1, 7, 4096])
torch.Size([1, 7, 4096])
torch.Size([1, 7, 4096])
torch.Size([1, 7, 4096])
torch.Size([1, 7, 4096])
torch.Size([1, 7, 4096])
torch.Size([1, 7, 4096])
torch.Size([1, 7, 4096])
torch.Size([1, 7, 4096])
torch.Size([1, 7, 4096])
torch.Size([1, 7, 4096])
torch.Size([1, 7, 4096])
torch.Size([1, 7, 4096])
torch.Size([1, 7, 4096])
torch.Size([1, 7, 4096])
torch.Size([1, 7, 4096])


In [7]:
len(hidden_states)

25

In [ ]:
token_embeddings = hidden_states[0]

In [3]:
names_1 = ["John", "Harry", "Andrew", "Lisa", "Mary"]
names_2 = ["Henry", "David", "Sophia", "Olivia", "Emma"]

cities_1 = ["London", "Paris", "Oslo", "Istanbul", "Beijing"]
cities_2 = ["Sydney", "Cairo", "Seoul", "Rome", "Prague"]

In [4]:
class DataSample_OneSentence():
    def __init__(self, name_1, city_1):
        self.name_1 = name_1
        self.city_1 = city_1
        self.context = name_1 + " travelled to " + city_1 + "."
        self.question_1 = "Where did " + name_1 + " travel to?"
        self.answer_1 = city_1
        
data_points_one_sentence = []
for name_1 in names_1 + names_2:
    for city_1 in cities_1 + cities_2:
        sample = DataSample_OneSentence(name_1, city_1)
        data_points_one_sentence.append(sample)

In [28]:
for name_1 in names_1 + names_2:
    input_ids = tokenizer.encode(name_1, return_tensors="pt")
    input_ids = input_ids.to(model.encoder.device)
    with torch.no_grad():
        encoder_outputs = model.encoder(input_ids, output_special=True, output_hidden_states=True)
    hidden_states = encoder_outputs.hidden_states
    for layer_hidden_states in hidden_states:
        print(layer_hidden_states.shape)

new code working-modeling_t5
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
new code working-modeling_t5
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2, 4096])
torch.Size([1, 2,

In [ ]:
all_hidden_states = encoder_out.hidden_states
token_embeddings = all_hidden_states[0][:, :-1, :]
nu = 3 * token_embeddings.std().item()
nu

In [53]:
tokens = names_1 + names_2 + ["travelled to"] + cities_1 + cities_2
tokens_str = " ".join(tokens)
input_ids = tokenizer.encode(tokens_str, return_tensors="pt")
#input_ids = input_ids.to(model.encoder.device)

with torch.no_grad():
    out_generation = model.generate(input_ids, max_new_tokens=1, return_dict_in_generate=True, output_scores=True )
out_generation.scores[0]

with torch.no_grad():
    encoder_out = model.encoder(input_ids, output_hidden_states=True)
all_hidden_states = encoder_out.hidden_states
token_embeddings = all_hidden_states[-1][:, :, :]

decoder_input_ids = tokenizer.encode('<pad>', add_special_tokens=False, return_tensors="pt")
with torch.no_grad():
    decoder_out = model.decoder(input_ids=decoder_input_ids, 
                            encoder_hidden_states=token_embeddings)
sequence_output = decoder_out[0]
with torch.no_grad():
    lm_logits = model.lm_head(sequence_output)
next_token_logits = lm_logits[:, -1, :]
next_tokens_scores = logits_processor(decoder_input_ids, next_token_logits )

new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5


In [54]:
model_out = model(decoder_input_ids=decoder_input_ids, 
                            encoder_outputs=encoder_out)

new code working-modeling_t5


In [55]:
out_generation.scores[0]

tensor([[-40.8957,  -5.4584, -11.6888,  ..., -39.5223, -40.7970, -40.1869]])

In [56]:
next_tokens_scores

tensor([[-40.8957,  -5.4584, -11.6888,  ..., -39.5223, -40.7970, -40.1869]],
       device='cuda:1')

In [57]:
next_token_logits

tensor([[-40.8957,  -5.4584, -11.6888,  ..., -39.5223, -40.7970, -40.1869]],
       device='cuda:1')

In [58]:
model_out.logits

tensor([[[-40.8957,  -5.4584, -11.6888,  ..., -39.5223, -40.7970, -40.1869]]],
       grad_fn=<ToCopyBackward0>)

In [34]:
all_hidden_states = encoder_out.hidden_states
token_embeddings = all_hidden_states[0][:, :-1, :]
nu = 3 * token_embeddings.std().item()
nu

23.895380973815918

tensor(0.9928, dtype=torch.float64)

In [29]:
B, T, d = encoder_out.last_hidden_state.shape

In [38]:
B, T, d = encoder_out.last_hidden_state.shape
model_outs = []
decoder_input_ids = tokenizer.encode('<pad>', add_special_tokens=False, return_tensors="pt")

for i in range(T):
    encoder_out_copy = encoder_out.copy()
    encoder_out_copy.last_hidden_state[0, i, :] += nu * torch.from_numpy(prng.randn(d))
    
    model_out = model(decoder_input_ids=decoder_input_ids, 
                    encoder_outputs=encoder_out_copy)
    
    model_outs.append(model_out)

new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5


In [43]:

next_token_scores = torch.nn.functional.softmax(
                    model_outs[0].logits, dim=-1
                )  # (batch_size * num_beams, vocab_size)


In [45]:
next_token_scores.max()

tensor(0.5108, grad_fn=<MaxBackward1>)

In [46]:
torch.argmax(next_token_scores)

tensor(24377)

In [47]:
tokenizer.decode(24377)

'Omni'

In [17]:
tokens = names_1 + names_2 + ["travelled to"] + cities_1 + cities_2
tokens_str = " ".join(tokens)
input_ids = tokenizer.encode(tokens_str, return_tensors="pt")
input_ids = input_ids.to(model.encoder.device)

out_generation = model.generate(input_ids, return_dict_in_generate=True, output_scores=True )
out_generation.scores[0]

encoder_out = model.encoder(input_ids, output_hidden_states=True)
all_hidden_states = encoder_out.hidden_states
token_embeddings = all_hidden_states[-1][:, :, :]
decoder_input_ids = tokenizer.encode('<pad>', add_special_tokens=False, return_tensors="pt").to(model.decoder.device)
decoder_out = model.decoder(input_ids=decoder_input_ids, 
                            encoder_hidden_states=token_embeddings)
sequence_output = decoder_out[0]
#sequence_output = sequence_output.cpu()
#sequence_output = sequence_output * (model.model_dim**-0.5)
sequence_output = sequence_output.to(model.decoder.device)
lm_logits = model.lm_head(sequence_output)
next_token_logits = lm_logits[:, -1, :]
next_tokens_scores = logits_processor(decoder_input_ids, next_token_logits )


new code working-utils
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5


/kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/generation/utils.py:1320: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


llama generation happening.
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
llama generation happening.
new code working-modeling_t5
llama generation happening.
new

NameError: name 'logits_processor' is not defined

In [9]:
sequence_output = decoder_out[0]

In [20]:
next_token_logits

tensor([[-40.8957,  -5.4584, -11.6888,  ..., -39.5223, -40.7970, -40.1869]],
       device='cuda:0', grad_fn=<SliceBackward0>)

In [21]:
out_generation.scores[0]


tensor([[-40.8957,  -5.4584, -11.6888,  ..., -39.5223, -40.7970, -40.1869]],
       device='cuda:0')

In [23]:
model_out.logits

tensor([[[-40.8957,  -5.4584, -11.6888,  ..., -39.5223, -40.7970, -40.1869]]],
       device='cuda:0', grad_fn=<UnsafeViewBackward0>)

In [25]:
model_out.logits[0] == out_generation.scores[0] == next_token_logits

RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [26]:
out_generation.scores[0] == next_token_logits

tensor([[True, True, True,  ..., True, True, True]], device='cuda:0')

In [11]:
sequence_output = sequence_output.to(model.decoder.device)

In [12]:
sequence_output

tensor([[[-0.8135,  0.1494,  0.1446,  ..., -0.0052,  0.2782, -0.1371]]],
       device='cuda:0', grad_fn=<ToCopyBackward0>)

In [15]:
sequence_output = sequence_output.cpu()

In [16]:
sequence_output

tensor([[[-0.8135,  0.1494,  0.1446,  ..., -0.0052,  0.2782, -0.1371]]],
       grad_fn=<ToCopyBackward0>)

In [13]:
lm_logits = model.lm_head(sequence_output)

In [14]:
lm_logits

tensor([[[-40.8957,  -5.4584, -11.6888,  ..., -39.5223, -40.7970, -40.1869]]],
       device='cuda:0', grad_fn=<UnsafeViewBackward0>)

In [294]:
model.decoder.device

device(type='cuda', index=0)

In [292]:
decoder_out = model.decoder(input_ids=decoder_input_ids, 
                            encoder_hidden_states=token_embeddings)
sequence_output = decoder_out[0]

new code working-modeling_t5


In [281]:
sequence_output

tensor([[[-0.1527,  0.1050, -0.0610,  ..., -0.0721,  0.2040,  0.0394]]],
       device='cuda:1', grad_fn=<MulBackward0>)

In [282]:
sequence_output = sequence_output * (model.model_dim**-0.5)

In [283]:
sequence_output

tensor([[[-0.0024,  0.0016, -0.0010,  ..., -0.0011,  0.0032,  0.0006]]],
       device='cuda:1', grad_fn=<MulBackward0>)

In [284]:
sequence_output = sequence_output.cpu()

In [285]:
sequence_output

tensor([[[-0.0024,  0.0016, -0.0010,  ..., -0.0011,  0.0032,  0.0006]]],
       grad_fn=<ToCopyBackward0>)

In [286]:
sequence_output = sequence_output.to(model.decoder.device)

In [287]:
sequence_output

tensor([[[-0.0024,  0.0016, -0.0010,  ..., -0.0011,  0.0032,  0.0006]]],
       device='cuda:0', grad_fn=<ToCopyBackward0>)

In [260]:
lm_logits = model.lm_head(sequence_output)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:1 and cuda:0! (when checking argument for argument mat2 in method wrapper_mm)

In [239]:
(model.model_dim**-0.5) * sequence_output

tensor([[[-0.0263,  0.0120, -0.0458,  ...,  0.0440,  0.0170, -0.0112]]],
       device='cuda:0', grad_fn=<MulBackward0>)

In [247]:
decoder_out[0]

tensor([[[-0.1816, -0.0095,  0.0301,  ...,  0.0225,  0.0094,  0.0319]]],
       device='cuda:1', grad_fn=<MulBackward0>)

In [253]:
sequence_output = sequence_output  * (model.model_dim**-0.5)

In [254]:
sequence_output

tensor([[[-0.0263,  0.0120, -0.0458,  ...,  0.0440,  0.0170, -0.0112]]],
       device='cuda:0', grad_fn=<MulBackward0>)

In [242]:
sequence_output

tensor([[[-1.6818,  0.7661, -2.9291,  ...,  2.8181,  1.0853, -0.7193]]],
       device='cuda:0', grad_fn=<ToCopyBackward0>)

In [218]:
decoder_input_ids

tensor([[0]], device='cuda:0')

In [219]:
out_generation.sequences

tensor([[    0, 23632,  5850, 11259, 17106,     3, 17084, 23729, 10977, 15447,
         10482,  1411, 11164, 31983,    35,  3445,  1411, 11164, 31983,    35]],
       device='cuda:0')

In [205]:
model_out = model(decoder_input_ids=decoder_input_ids, 
                            encoder_outputs=encoder_out)

new code working-modeling_t5


In [206]:
model_out.keys()

odict_keys(['logits', 'past_key_values', 'decoder_attentions', 'cross_attentions', 'encoder_last_hidden_state', 'encoder_hidden_states', 'encoder_attentions'])

In [220]:
model_out.logits

tensor([[[-832.8788,  264.9427,  294.8038,  ..., -733.3184, -860.7432,
          -800.1171]]], device='cuda:0', grad_fn=<UnsafeViewBackward0>)

In [228]:
model.config.tie_word_embeddings

False

In [291]:
out_generation.scores[0]

tensor([[-186694.4219,   48423.9883,   67818.1875,  ..., -168093.0938,
         -194783.8750, -181539.8906]], device='cuda:0')

In [289]:
next_tokens_scores

tensor([[-54.5876, -14.7907, -13.3360,  ..., -53.9566, -54.0444, -54.2783]],
       device='cuda:0', grad_fn=<SliceBackward0>)

In [290]:
next_token_logits

tensor([[-54.5876, -14.7907, -13.3360,  ..., -53.9566, -54.0444, -54.2783]],
       device='cuda:0', grad_fn=<SliceBackward0>)

In [293]:
sequence_output

tensor([[[-0.1527,  0.1050, -0.0610,  ..., -0.0721,  0.2040,  0.0394]]],
       device='cuda:1', grad_fn=<MulBackward0>)

In [232]:
next_tokens_scores = logits_processor(input_ids, next_token_logits )
next_tokens_scores

tensor([[-380.7138,   96.3923,  116.1752,  ..., -330.2903, -398.2517,
         -364.1730]], device='cuda:0', grad_fn=<SliceBackward0>)

In [185]:
model.model_dim

4096

In [ ]:
sequence_output = sequence_output * (model.model_dim**-0.5)

In [183]:
next_tokens_scores.shape

torch.Size([1, 32128])

In [216]:
next_tokens_scores

tensor([[-380.7138,   96.3923,  116.1752,  ..., -330.2903, -398.2517,
         -364.1730]], device='cuda:0', grad_fn=<SliceBackward0>)

In [160]:
out_generation.sequences[0]

tensor([    0, 23632,  5850, 11259, 17106, 16580,  5823, 25216, 23844, 21888,
        23595, 14365, 10340, 18820, 17800, 17575, 22976,  7423, 16225, 31780],
       device='cuda:0')

In [159]:
len(out_generation.scores)

19

In [163]:
out_generation.scores[1].shape

torch.Size([1, 32128])

In [128]:
out_generation.keys()

odict_keys(['sequences', 'scores', 'encoder_attentions', 'decoder_attentions', 'cross_attentions'])

In [88]:
out.keys()

odict_keys(['last_hidden_state', 'hidden_states', 'attentions'])

In [164]:
out.last_hidden_state == token_embeddings

tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]]], device='cuda:1')

In [92]:
token_embeddings.shape

torch.Size([1, 23, 4096])

In [165]:
decoder_input_ids = tokenizer.encode('<pad>', add_special_tokens=False, return_tensors="pt").to(model.decoder.device)

In [166]:
decoder_out = model.decoder(input_ids=decoder_input_ids, 
                            encoder_hidden_states=out.last_hidden_state)


new code working-modeling_t5


In [167]:
decoder_out.keys()

odict_keys(['last_hidden_state', 'past_key_values', 'attentions', 'cross_attentions'])

In [168]:
(decoder_out[0] == decoder_out.last_hidden_state).sum()

tensor(4096, device='cuda:1')

In [169]:
decoder_out.last_hidden_state.shape

torch.Size([1, 1, 4096])

In [170]:
lm_logits = model.lm_head(decoder_out[0].to(model.decoder.device))

In [171]:
lm_logits.shape

torch.Size([1, 1, 32128])

In [173]:
next_token_logits = lm_logits[:, -1, :]

In [174]:
next_tokens_scores = logits_processor(decoder_input_ids, next_token_logits )

In [180]:
next_tokens_scores

tensor([[-1854.4319,   650.3324,   974.5194,  ..., -1707.0854, -1870.9194,
         -1762.8221]], device='cuda:0', grad_fn=<SliceBackward0>)

In [179]:
next_tokens_scores == next_token_logits

tensor([[True, True, True,  ..., True, True, True]], device='cuda:0')

In [177]:
out_generation.scores[0]

tensor([[  7.8130, -78.9499, -40.4690,  ...,  32.9601,  -9.1687,  20.1957]],
       device='cuda:0')

In [187]:
next_tokens_scores == out_generation.scores[0]

tensor([[False, False, False,  ..., False, False, False]], device='cuda:0')

In [11]:
import sys
sys.path.append("/scratch/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/generation")
sys.path.append("/scratch/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers")
sys.path.append("/scratch/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages/transformers/generation/utils.py")
from logits_process import LogitsProcessorList

ImportError: attempted relative import with no known parent package

In [51]:
from transformers.generation.logits_process import LogitsProcessorList

In [143]:
sys.path = ['/scratch/users/bozyurt20/hpc_run/Sherlock Holmes/Adding Noise',
 '/kuacc/users/bozyurt20/.conda/envs/hf/lib/python38.zip',
 '/kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8',
 '/kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/lib-dynload',
 '',
 '/kuacc/users/bozyurt20/.conda/envs/hf/lib/python3.8/site-packages',
 '/scratch/users/bozyurt20/hpc_run/llama',
 '/scratch/users/bozyurt20/hpc_run/memformers',
 '/scratch/users/bozyurt20/hpc_run/TorchFly',
 '/scratch/users/bozyurt20/hpc_run/utilities',
 '/scratch/users/bozyurt20/hpc_run/blobs/']

In [52]:

logits_processor = LogitsProcessorList()

In [150]:
next_tokens_scores = logits_processor(input_ids, lm_logits.squeeze(0))

In [151]:
next_tokens_scores.shape

torch.Size([1, 32128])

In [153]:
next_tokens_scores == out_generation.scores[0]

tensor([[False, False, False,  ..., False, False, False]], device='cuda:0')

In [ ]:
decoder_outputs = self.decoder(
        input_ids=decoder_input_ids,
        attention_mask=decoder_attention_mask,
        inputs_embeds=decoder_inputs_embeds,
        past_key_values=past_key_values,
        encoder_hidden_states=hidden_states,
        encoder_attention_mask=attention_mask,
        head_mask=decoder_head_mask,
        cross_attn_head_mask=cross_attn_head_mask,
        use_cache=use_cache,
        output_attentions=output_attentions,
        output_hidden_states=output_hidden_states,
        return_dict=return_dict,
    )

In [81]:
tokenizer.tokenize(tokens_str, add_special_tokens=True)

['▁John',
 '▁Harry',
 '▁Andrew',
 '▁Lisa',
 '▁Mary',
 '▁Henry',
 '▁David',
 '▁Sophia',
 '▁Olivia',
 '▁Emma',
 '▁',
 'travelled',
 '▁to',
 '▁London',
 '▁Paris',
 '▁Oslo',
 '▁Istanbul',
 '▁Beijing',
 '▁Sydney',
 '▁Cairo',
 '▁Seoul',
 '▁Rome',
 '▁Prague',
 '</s>']

In [59]:
token_embeddings.std()

tensor(7.8574, device='cuda:0', grad_fn=<StdBackward0>)

In [58]:
token_embeddings.std(dim=1).mean()

tensor(7.7434, device='cuda:0', grad_fn=<MeanBackward0>)

In [47]:
all_hidden_states[0][:, 9, :] == emma_hidden_state[:, 0, :]

tensor([[True, True, True,  ..., True, True, True]], device='cuda:0')

In [46]:
emma_hidden_state.shape

torch.Size([1, 2, 4096])

In [ ]:
all_hidde

In [68]:
x = torch.rand(3,4)
y = torch.rand(3,4)

In [69]:
torch.cat((x, y)).shape

torch.Size([6, 4])